In [1]:
import os

# relevant only for this notebook
os.chdir("..")

import MLUtils.evaluation.FeatureSelection as FeatureSelection

%load_ext autoreload
%autoreload 2

In [2]:
from sklearn import datasets
iris = datasets.load_iris(as_frame=True)
iris_df = iris.frame
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
data_df = iris_df.drop(columns='target')
label_df = iris_df['target']

# Variance Feature Selection

In [13]:
selected_features_indices, selected_columns = FeatureSelection.get_variance_features(data_df, variance_threshold=0.05)

VarianceThreshold selected 3 out of 4 features


In [5]:
type(selected_features_indices)

numpy.ndarray

# Univariate feature selection

In [10]:
from sklearn.feature_selection import f_classif, mutual_info_classif

univariate_methods_list = [f_classif, mutual_info_classif]
num_features_to_select_by_score = 3
is_num_is_max = True
num_features_to_select_total = 2

selected_univariate_indices = FeatureSelection.get_univariate_feature_indices(data_df=data_df,
                                                                              label_df=label_df,
                                                                              univariate_methods_list=univariate_methods_list,
                                                                              num_features_to_select_by_score=num_features_to_select_by_score,
                                                                              num_features_to_select_total=num_features_to_select_total,
                                                                              is_num_is_max=is_num_is_max)

Evaluating scores
Computing ranks
Custom selection selected 2 out of 4 features


# Multivariate feature selection

In [11]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression



multivariate_features_indices, columns = \
FeatureSelection.get_multivariate_feature_indices(data_df=data_df,
                                                  label_df=label_df,
                                                  multivariate_methods_dict=multivariate_methods_dict,
                                                  num_features_to_select_by_score=num_features_to_select_by_score,
                                                  num_features_to_select_total=num_features_to_select_total,
                                                  is_num_is_max=False,
                                                  cross_validation=skf)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
Features: 1/4[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
Features: 2/4[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 3/4[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 4/4

Running sequential features selection
Running classifier number 1
Starting Decision Tree multivariate feature selection process
Evaluating fold 0
Evaluating fold 1
Evaluating fold 2
Computing ranks
Custom selection selected 2 out of 4 features


# Combination

In [22]:
iris = datasets.load_iris(as_frame=True)
iris_df = iris.frame
data_df = iris_df.drop(columns='target')
label_df = iris_df['target']

print("Original feature indices\n", list(range(data_df.shape[1])))
print("Original features\n", data_df.columns)

# univariante
univariate_methods_list = [f_classif, mutual_info_classif]
num_features_to_select_by_score = 3
is_num_is_max = True
num_features_to_select_total = 3

selected_univariate_indices, univariate_columns = \
FeatureSelection.get_univariate_feature_indices(data_df=data_df,
                                   label_df=label_df,
                                   univariate_methods_list=univariate_methods_list,
                                   num_features_to_select_by_score=num_features_to_select_by_score,
                                   num_features_to_select_total=num_features_to_select_total,
                                   is_num_is_max=is_num_is_max)

print(univariate_columns)
print(selected_univariate_indices)

univariate_features_df = data_df.iloc[:, selected_univariate_indices]

# multivariate
num_features_to_select_by_score = 3
num_features_to_select_total = 2
skf = KFold(n_splits=3)
multivariate_methods_dict = {
    "sfs":
        {
            "estimator": [LogisticRegression()],
            "forward": True,
            "n_jobs": -1,
            "k_features": 'parsimonious'
        },
    "decision_tree": {}
}

multivariate_features_indices, multivariatecolumns = \
FeatureSelection.get_multivariate_feature_indices(data_df=univariate_features_df,
                                     label_df=label_df,
                                     multivariate_methods_dict=multivariate_methods_dict,
                                     num_features_to_select_by_score=num_features_to_select_by_score,
                                     num_features_to_select_total=num_features_to_select_total,
                                     is_num_is_max=False,
                                     cross_validation=skf)

print(multivariatecolumns)
print(multivariate_features_indices)

multivariate_features_df = data_df.iloc[:, multivariate_features_indices]

selected_features_indices, selected_columns = \
FeatureSelection.get_variance_features(multivariate_features_df, variance_threshold=0.05)

print(selected_columns)
print(selected_features_indices)

variance_data_df = data_df.iloc[:, selected_features_indices]

Original feature indices
 [0, 1, 2, 3]
Original features
 Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)'],
      dtype='object')
Evaluating univariate scores
Computing ranks
Custom selection selected 3 out of 4 features
['petal length (cm)', 'petal width (cm)', 'sepal length (cm)']
[2, 3, 0]
Evaluating multivariate scores
Running sequential features selection
Running classifier number 1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.1s finished
Features: 1/3[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.4s finished
Features: 2/3[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished
Features: 3/3

Starting Decision Tree multivariate feature selection process
Evaluating fold 0
Evaluating fold 1
Evaluating fold 2
Computing ranks
Custom selection selected 3 out of 3 features
['petal length (cm)', 'petal width (cm)', 'sepal length (cm)']
[0, 1, 2]
VarianceThreshold selected 2 out of 3 features
['sepal length (cm)', 'petal length (cm)']
[0, 2]
